> Required Imports

In [1]:
import pandas as pd
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Conv2D, MaxPooling2D
from keras.models import Sequential
import numpy as np
from keras.optimizers import Adam
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from os.path import exists
from config import CONFIG
import warnings
warnings.filterwarnings('ignore')

> Get the recognition trained model, and get the convolutions performed, and use that to build a model that only convolutes the input

In [ ]:
# Load trained recognition model
loaded_model = keras.models.load_model(CONFIG.get("files", "recog_model"))

# Get layerwise weights
weights = []
for layr in loaded_model.layers:
    weights.append(layr.get_weights())

# Build only the convolutional part of the network and assign the trained weights
model = Sequential()
model.add(Conv2D(input_shape=(128,171,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))

for i, layr in enumerate(model.layers):
    model.layers[i].set_weights(weights[i])
    model.layers[i].trainable = False

In [ ]:
df = pd.read_pickle(CONFIG.get("files", "base_data_spectro"))
model.predict(np.array([df['ps'][0].reshape((128,171,1))]), verbose=0)[0]

> Convolute the spectrograms to get the input for the decoders, and save it before further training

In [ ]:
def conv(ps):
    # Convolute the Spectrogram to get a convoluted 4096 dimensional output (input to decoder)
    return model.predict(np.array([ps.reshape((128,171,1))]), verbose=0)[0]

def flatten(ps):
    # Flatten the spectrogram to get a 1D representation representing the output
    return ps.flatten()

> Load saved X and Y for training the Decoder, or create the X and Y by convoluting and flattening the spectrograms (respectively)

In [ ]:
reconst_train_test = CONFIG.get("files", "reconst_train_test_data")

if exists(reconst_train_test):
    with open(reconst_train_test, 'rb') as f:
        x_tr = np.load(f, allow_pickle=True)
        y_tr = np.load(f, allow_pickle=True)
else:
    x_tr = np.array(df['ps'].map(conv).values)
    y_tr = np.array(df['ps'].map(flatten).values)
    with open(reconst_train_test, 'wb') as f:
        np.save(f, x_tr)
        np.save(f, y_tr)

> Training

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.25, shuffle=True)

x_train = np.array([x.reshape( (4096, 1) ) for x in x_train])
x_test = np.array([x.reshape( (4096, 1) ) for x in x_test])

In [5]:
pixels = 1216
num_networks = 18
input_shape=x_train[0].shape

In [6]:
for i_nn in range(num_networks):
    y_train_i = tf.convert_to_tensor([x[pixels*i_nn:pixels*(i_nn+1)].tolist() for x in y_train])
    y_test_i = tf.convert_to_tensor([x[pixels*i_nn:pixels*(i_nn+1)].tolist() for x in y_test])
    
    tf.keras.backend.clear_session()
    
    model2 = Sequential()
    model2.add(Conv1D(input_shape=(4096,1),filters=32,kernel_size=3,padding="same", activation="relu"))
    model2.add(Conv1D(filters=32,kernel_size=3,padding="same", activation="relu"))
    model2.add(MaxPooling1D(pool_size=2,strides=2))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(MaxPooling1D(pool_size=2,strides=2))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(MaxPooling1D(pool_size=2,strides=2))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model2.add(MaxPooling1D(pool_size=2,strides=2))
    model2.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model2.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model2.add(MaxPooling1D(pool_size=2,strides=2))
    
    model2.add(Flatten())
    model2.add(Dense(units=4096,activation="relu"))
    model2.add(Dense(units=y_train_i[0].shape[0])) # no activation is linear
    
    model2.compile(
        optimizer = Adam(learning_rate=0.001),
        loss = "mse",
        metrics = ['mse'])
    
    history= model2.fit(
        x=x_train,
        y=y_train_i,
        epochs=15,
        batch_size=1000,
        verbose=0,
        validation_data= (x_test, y_test_i))

    score = model2.evaluate(
        x=x_test,
        y=y_test)

    print(f"\nMODEL-{i_nn}")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    model2.save( CONFIG.get("files", "reconst_models") + f'model_{i_nn}')

Metal device set to: Apple M2


2022-12-04 16:11:40.781072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-04 16:11:40.781173: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/15


2022-12-04 16:11:41.178452: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-04 16:11:41.647416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 884.2518 - mse: 884.2518

...

4/4 [==============================] - ETA: 0s - loss: 6.5969e-05 - mse: 6.5969e-05

2022-12-04 16:31:20.481159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 5s 1s/step - loss: 6.5969e-05 - mse: 6.5969e-05 - val_loss: 1.2940e-04 - val_mse: 1.2940e-04
Epoch 2/15
4/4 [==============================] - 4s 1s/step - loss: 4.8810e-05 - mse: 4.8810e-05 - val_loss: 1.2781e-04 - val_mse: 1.2781e-04
Epoch 3/15
4/4 [==============================] - 4s 1s/step - loss: 4.7442e-05 - mse: 4.7442e-05 - val_loss: 1.2730e-04 - val_mse: 1.2730e-04
Epoch 4/15
4/4 [==============================] - 4s 1s/step - loss: 4.7175e-05 - mse: 4.7175e-05 - val_loss: 1.2735e-04 - val_mse: 1.2735e-04
Epoch 5/15
4/4 [==============================] - 4s 1s/step - loss: 4.7213e-05 - mse: 4.7213e-05 - val_loss: 1.2730e-04 - val_mse: 1.2730e-04
Epoch 6/15
4/4 [==============================] - 4s 1s/step - loss: 4.7127e-05 - mse: 4.7127e-05 - val_loss: 1.2720e-04 - val_mse: 1.2720e-04
Epoch 7/15
4/4 [==============================] - 4s 1s/step - loss: 4.7036e-05 - mse: 4.7036e-05 - val_loss: 1.2716e-04 - val_mse: 1.2716e-04
Epoch 8/15

INFO:tensorflow:Assets written to: ../data/reconstruction_models/model_17/assets


INFO:tensorflow:Assets written to: ../data/reconstruction_models/model_17/assets
